# DeepSphere using ModelNet40 dataset
### Benchmark with Cohen method S2CNN[[1]](http://arxiv.org/abs/1801.10130) and Esteves method[[2]](http://arxiv.org/abs/1711.06721) and others spherical CNNs
Multi-class classification of 3D objects, using the interesting property of rotation equivariance.

The 3D objects are projected on a unit sphere.
Cohen and Esteves use equiangular sampling, while our method use a HEAlpix sampling

Several features are collected:
* projection ray length (from sphere border to intersection [0, 2])
* cos/sin with surface normal
* same features using the convex hull of the 3D object

## 0.1 Load libs

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import shutil
import sys
sys.path.append('../../')

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to chosen GPU to use, nothing if work on CPU

import numpy as np
import time
import matplotlib.pyplot as plt
import healpy as hp

In [ ]:
from deepsphere import models, experiment_helper, plot, utils
from deepsphere.data import LabeledDatasetWithNoise, LabeledDataset
import hyperparameters

from load_MN40 import plot_healpix_projection, ModelNet40DatasetTF, ModelNet40DatasetCache

## 0.2 Define parameters

In [ ]:
Nside = 32
exp='norot' # in ['rot', 'norot', 'pert', 'Z']
datapath = '../../../data/ModelNet40/' # localisation of the .OFF files
proc_path = datapath[1:]

In [ ]:
augmentation = 1        # number of element per file (1 = no augmentation of dataset)
nfeat = 6

#### Test projection

In [ ]:
import trimesh
dataset = '/train/'
clas = 'airplane'
mesh = trimesh.load_mesh(datapath+clas+dataset+clas+"_0119.off")
mesh.remove_degenerate_faces()
mesh.remove_duplicate_faces()
mesh.show()

In [ ]:
from load_MN40 import rotmat, rnd_rot

In [ ]:
mesh = trimesh.load_mesh(datapath+clas+dataset+clas+"_0338.off")
mesh.remove_degenerate_faces()
mesh.fix_normals()
mesh.fill_holes()
mesh.remove_duplicate_faces()
mesh.remove_infinite_values()
mesh.remove_unreferenced_vertices()

mesh.apply_translation(-mesh.centroid)
r = np.max(np.linalg.norm(mesh.vertices, axis=-1))
mesh.apply_scale(1 / r)

mesh.apply_transform(rnd_rot(z=0, c=0))

r = np.max(np.linalg.norm(mesh.vertices, axis=-1))
mesh.apply_scale(0.99 / r)
mesh.remove_degenerate_faces()
mesh.fix_normals()
mesh.fill_holes()
mesh.remove_duplicate_faces()
mesh.remove_infinite_values()
mesh.remove_unreferenced_vertices()
mesh.show()


In [ ]:
dataset = '/train/'
_class = 'table'
plot_healpix_projection(datapath+_class+dataset+_class+"_0001.off", 32, rotp = False, rot = (90,0,0))

## 1 Load dataset

In [ ]:
train_rot_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                              experiment='deepsphere_rot_notr')


In [ ]:
train_rot_tr_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                           experiment='deepsphere_rot')


In [ ]:
train_tr_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                       experiment='deepsphere')


In [ ]:
train_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=1, nfile=None, 
                                          experiment='deepsphere_notr')


In [ ]:
train_Z_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                         experiment='deepsphere_Z')


Better to keep validation and testing set in RAM, but not always possible

In [ ]:
test_tr_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None)

In [ ]:
test_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, nfeat=nfeat, augmentation=1, nfile=None,
                                        experiment='deepsphere_notr')

In [ ]:
test_rot_tr_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, 
                                       nfeat=nfeat, experiment='deepsphere_rot', augmentation=3, nfile=None)

In [ ]:
test_rot_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, 
                                       nfeat=nfeat, experiment='deepsphere_rot_notr', augmentation=3, nfile=None)

In [ ]:
test_Z_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, 
                                       nfeat=nfeat, experiment='deepsphere_Z', augmentation=3, nfile=None)

Try do make a tensorflow dataset object

In [ ]:
experiment = 'deepsphere'+('_rot' if exp == 'rot' else '')+('_Z' if exp == 'Z' else '')+('_notr' if 'pert' not in exp and exp != 'Z' else '')
train_TFDataset = ModelNet40DatasetTF(datapath, 'train', nside=Nside,
                                      nfeat=nfeat, augmentation=augmentation, nfile=None, experiment=experiment)

In [ ]:
train_TFDataset.N

### 1.1 compute stats and test dataset

In [ ]:
from load_MN40 import compute_mean_std

In [ ]:
compute_mean_std(train_dataset, 'train', datapath, Nside)

In [ ]:
dataset = train_TFDataset.get_tf_dataset(32)

In [ ]:
import tensorflow as tf
from tqdm import tqdm

#dataset = tf_dataset_file(datapath, dataset, file_pattern, 32, Nside, augmentation)
data_next = dataset.make_one_shot_iterator().get_next()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
steps = train_TFDataset.N // 32 + 1
cm = plt.cm.RdBu_r
cm.set_under('w')
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in tqdm(range(steps)):
        data, label = sess.run(data_next)
        im1 = data[0,:,0]
        cmin = np.nanmin(im1)
        cmax = np.nanmax(im1)
        hp.orthview(im1, rot=(0,0,0), title=train_TFDataset.classes[label[0]], nest=True, cmap=cm, min=cmin, max=cmax)
        plt.figure()
        if i > 2:
            suffix = train_TFDataset.classes[label[0]]
            break
#     except tf.errors.OutOfRangeError:
#         print("Done") 

In [ ]:
def transform(data, phi=None, theta=None):
    batch_size, npix, nfeat = data.shape
    if theta is None or phi is None:
        phi = np.random.rand() * 2 * np.pi
        theta = np.random.rand() * np.pi
    nside = hp.npix2nside(npix)

    # Get theta, phi for non-rotated map
    t,p = hp.pix2ang(nside, np.arange(npix), nest=True) #theta, phi

    # Define a rotator
    r = hp.Rotator(deg=False, rot=[phi, theta])

    # Get theta, phi under rotated co-ordinates
    trot, prot = r(t,p)

    # Interpolate map onto these co-ordinates
    new_data = np.zeros(data.shape)
    for b in range(batch_size):
        for f in range(nfeat):
            new_data[b,:,f] = hp.get_interp_val(data[b,:,f], trot, prot, nest=True)

    return new_data

In [ ]:
def transform_equator(data):
    return transform(data, 0, np.pi/2).astype(np.float32)

In [ ]:
def transform_shift(data):
    """
    90° rotation around poles (natural Z-axis)
    """
    batch_size, npix, nfeat = data.shape
    new_data = data.copy()
    nside = hp.npix2nside(npix)
    theta, _ = hp.pix2ang(nside, range(npix))
    theta_u = np.unique(theta)
    for b in range(batch_size):
        for f in range(nfeat):
            new_data[b, :, f] = hp.reorder(data[b, :, f], n2r=True)
            for t in theta_u:
                ligne_ind = np.where(theta==t)[0]
                ligne_ind_roll = np.roll(ligne_ind, len(ligne_ind)//4)
                new_data[b,ligne_ind_roll,f] = new_data[b,ligne_ind,f]
            new_data[b, :, f] = hp.reorder(new_data[b, :, f], r2n = True)
    return new_data

In [ ]:
def transform_inverse(data):
    """
    180° rotation around X-axis
    """
    batch_size, npix, nfeat = data.shape
    data_c = data.copy()
    new_data = data.copy()
    new_data[:] = -10
    nside = hp.npix2nside(npix)
    theta, _ = hp.pix2ang(nside, range(npix))
    theta_u = np.unique(theta)
    for b in range(batch_size):
        for f in range(nfeat):
            data_c[b, :, f] = hp.reorder(data[b, :, f], n2r=True)
            for i, (t, t_end) in enumerate(zip(theta_u, theta_u[::-1])):
                ligne_ind = np.where(theta==t)[0]
                ligne_ind_roll = np.where(theta==t_end)[0][::-1]
                if i > len(theta_u)/4 and i < len(theta_u)*3/4:
                    ligne_ind_roll = np.roll(ligne_ind_roll, (i+1)%2)
                new_data[b,ligne_ind_roll,f] = data_c[b,ligne_ind,f]
            new_data[b, :, f] = hp.reorder(new_data[b, :, f], r2n = True)
    return new_data

In [ ]:
hp.orthview(im1, rot=(0,0,0), title=suffix, nest=True, cmap=cm, min=cmin, max=cmax)
plt.figure()
im2 = transform_shift(im1[np.newaxis,:,np.newaxis])
hp.orthview(im2[0,:,0], rot=(0,0,0), title=suffix, nest=True, cmap=cm, min=cmin, max=cmax)
plt.figure()
im2 = transform_inverse(im2)
hp.orthview(im2[0,:,0], rot=(0,0,0), title=suffix, nest=True, cmap=cm, min=cmin, max=cmax)

### 1.2 create dataset

In [ ]:
from tqdm import tqdm
size = 1 # 32
steps = test_dataset.N // size + 1
data_iter = test_dataset.iter(size)
cm = plt.cm.RdBu_r
cm.set_under('w')
for i in tqdm(range(steps)):
    data, label = next(data_iter)
    im1 = data[0,:,0]
#     if np.std(im1)>2:
#         print(np.std(im1))
#     cmin = np.nanmin(im1)
#     cmax = np.nanmax(im1)
#     hp.orthview(im1, rot=(0,0,0), title=test_dataset.classes[label[0]], nest=True, cmap=cm, min=cmin, max=cmax)
#     plt.figure()
#     if i > 10:
#         break


## 1.3 Informations

Shuffle the training dataset and print the classes distribution

In [ ]:
nclass = train_TFDataset.nclass
num_elem = train_TFDataset.N
print('number of class:',nclass,'\nnumber of elements:',num_elem)

## 2 Classification using DeepSphere

In [ ]:
EXP_NAME = 'MN40_{}_{}feat_{}aug_{}sides'.format(exp, nfeat, augmentation, Nside)

Load model with hyperparameters chosen.
For each experiment, a new EXP_NAME is chosen, and new hyperparameters are store.
All informations are present 'DeepSphere/Shrec17/experiments.md'
The fastest way to reproduce an experiment is to revert to the commit of the experiment to load the correct files and notebook

Adding a layer in the fully connected can be beneficial

In [ ]:
params = hyperparameters.get_params_mn40(train_TFDataset.N, EXP_NAME, Nside, nclass, 
                                                  nfeat_in=nfeat, architecture='CNN')  # get_params_shrec17_optim
params["tf_dataset"] = train_TFDataset.get_tf_dataset(params["batch_size"])
#params["std"] = [0.001, 0.005, 0.0125, 0.05, 0.15, 0.5] # [0.00002, 0.0002, 0.001, 0.005, 0.0125, 0.05] # best std for nside = 32
#params["full"] = [True]*6
model = models.deepsphere(**params)

In [ ]:
shutil.rmtree('../../summaries/{}/'.format(EXP_NAME), ignore_errors=True)
shutil.rmtree('../../checkpoints/{}/'.format(EXP_NAME), ignore_errors=True)

Find a correct learning rate

In [ ]:
# backup = params.copy()

# params, learning_rate = utils.test_learning_rates(params, train_TFDataset.N, 1e-6, 1e-1, num_epochs=20)

# shutil.rmtree('summaries/{}/'.format(params['dir_name']), ignore_errors=True)
# shutil.rmtree('checkpoints/{}/'.format(params['dir_name']), ignore_errors=True)

# model = models.deepsphere(**params)
# _, loss_validation, _, _ = model.fit(train_TFDataset, val_dataset, use_tf_dataset=True, cache=True)

# params.update(backup)

# plt.semilogx(learning_rate, loss_validation, '.-')

In [ ]:
# shutil.rmtree('summaries/lr_finder/', ignore_errors=True)
# shutil.rmtree('checkpoints/lr_finder/', ignore_errors=True)

0.9 seems to be a good learning rate for SGD with current parameters

## 2.2 Train Network

In [ ]:
print("the number of parameters in the model is: {:,}".format(model.get_nbr_var()))

In [ ]:
accuracy_validation, loss_validation, loss_training, t_step, t_batch = model.fit(train_TFDataset, 
                                                                                 test_dataset, 
                                                                                 use_tf_dataset=True, cache=True)

In [ ]:
plot.plot_loss(loss_training, loss_validation, t_step, params['eval_frequency'])

Remarks

In [ ]:
model.evaluate(train_rot_dataset, None, cache=True)

In [ ]:
model.evaluate(train_rot_tr_dataset, None, cache=True)

In [ ]:
model.evaluate(train_dataset, None, cache=True)

In [ ]:
model.evaluate(train_tr_dataset, None, cache=True)

In [ ]:
model.evaluate(train_Z_dataset, None, cache=True)

## 3 test network

In [ ]:
model.evaluate(test_dataset, None, cache=True)

In [ ]:
test_dataset.set_transform(transform_shift)
print(model.evaluate(test_dataset, None, cache=True))
test_dataset.set_transform(None)

In [ ]:
model.evaluate(test_rot_dataset, None, cache=True)

In [ ]:
model.evaluate(test_rot_tr_dataset, None, cache=True)

In [ ]:
model.evaluate(test_tr_dataset, None, cache=True)

In [ ]:
model.evaluate(test_Z_dataset, None, cache=True)

### 3.1 exploration of results

In [ ]:
predictions, loss = model.predict(test_dataset, None, cache=True)
print(loss)

In [ ]:
predictions_rot, loss = model.predict(test_rot_dataset, None, cache=True)
print(loss)

In [ ]:
labels_test = test_dataset.get_labels()

In [ ]:
### class attribution for "flower_pot" shapes

from collections import Counter
from sklearn.metrics import accuracy_score
# hist = Counter(predictions)
hist = Counter(labels_test)
tot = 0
accuracy_class = np.empty((40,))
for _class, nb in sorted(hist.items()):
    if _class == 15:
        for pred in predictions[tot:tot+nb]:
            print(test_dataset.classes[int(pred)])
    accuracy_class[int(_class)] = accuracy_score(labels_test[tot:tot+nb], predictions[tot:tot+nb])*100
    tot += nb

In [ ]:
### accuracy per class
plt.plot(accuracy_class, 'o')

In [ ]:
test_dataset.classes[np.argmax(accuracy_class)]

In [ ]:
test_dataset.classes[np.argmin(accuracy_class)]

Add rotation perturbations

In [ ]:
labels_3_test = test_rot_dataset.get_labels()

In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score
# hist = Counter(predictions)
hist = Counter(labels_3_test)
tot = 0
accuracy_class = np.empty((40,))
for _class, nb in sorted(hist.items()):
    accuracy_class[int(_class)] = accuracy_score(labels_3_test[tot:tot+nb], predictions_rot[tot:tot+nb])*100
    tot += nb

In [ ]:
plt.plot(accuracy_class, 'o')

Accuracy per class seems similar, and undistinguishable classes worsens

### 3.2 evolution of logits for a specific class

In [ ]:
files = test_dataset.files
files = [file for file in files if 'flower_pot' in file]
files = files[:64]
batch_1 = np.vstack(test_dataset.get_npy_file(files))
batch_1_rot = np.vstack(test_rot_dataset.get_npy_file(files))

In [ ]:
test_dataset.set_transform(transform_shift)
batch_1_shift = np.vstack(test_dataset.get_npy_file(files))
test_dataset.set_transform(None)

In [ ]:
probs = model.probs(batch_1, 40)
probs_shift = model.probs(batch_1_shift, 40)
probs_rot = model.probs(batch_1_rot, 40)

In [ ]:
obj = 17
plt.plot(probs[obj,:], 'o', markersize=10, label='normal')
# plt.plot(probs_shift[obj,:], 'o', label='90 shift')
plt.plot(probs_rot[3*obj:3*obj+3, :].T, 'o', markersize=4, label = 'rotx')
plt.legend()

In [ ]:
class_max = np.argmax(probs[obj,:])
test_dataset.classes[class_max]

In [ ]:
class_max = np.argmax(probs_rot[3*obj:3*obj+3, :].mean(axis=0))
test_dataset.classes[class_max]

In [ ]:
cm = plt.cm.RdBu_r
cm.set_under('w')
cmin = np.min(batch_1[:,:,0])
cmax = np.max(batch_1[:,:,0])
hp.orthview(batch_1[obj,:,0], rot=(0,0,0), title=files[obj], nest=True, cmap=cm, min=cmin, max=cmax)

In [ ]:
for i in range(3):
    plt.figure()
    hp.orthview(batch_1_rot[3*obj+i,:,0], rot=(0,0,0), title=files[obj], nest=True, cmap=cm, min=cmin, max=cmax)

Class distribution

In [ ]:
def _print_histogram(nclass, labels_train, labels_min=None, ylim=None):
    if labels_train is None:
        return
    import matplotlib.pyplot as plt
    from collections import Counter
    hist_train=Counter(labels_train)
    if labels_min is not None:
        hist_min = Counter(labels_min)
        hist_temp = hist_train - hist_min
        hist_min = hist_min - hist_train
        hist_train = hist_temp + hist_min
#         for i in range(self.nclass):
#             hist_train.append(np.sum(labels_train == i))
    labels, values = zip(*hist_train.items())
    indexes = np.asarray(labels)
#     miss = set(indexes) - set(labels)
#     if len(miss) is not 0:
#         hist_train.update({elem:0 for elem in miss})
#     labels, values = zip(*hist_train.items())
    width = 1
    plt.bar(labels, values, width)
    plt.title("labels distribution")
    plt.ylim(0,ylim)
    #plt.xticks(indexes + width * 0.5, labels)
    plt.show()

In [ ]:
_print_histogram(40, labels_test)
_print_histogram(40, predictions)
_print_histogram(40, labels_test, predictions, ylim=200)

In [ ]:
from collections import Counter
tot = predictions.shape[0]
hist = Counter(predictions)
hist.subtract(Counter(labels_test))
p_tot = 0
for _class, nb in hist.most_common():
    percent = 100*nb/Counter(labels_test)[_class]
    p_tot += percent
    print("{:2.0f}".format(_class), test_rot_dataset.classes[int(_class)], "{:.2f}".format(percent))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
plt.imshow(confusion_matrix(labels_test, predictions, range(40)), cmap = plt.cm.gist_heat_r)